<a href="https://colab.research.google.com/github/Mayank-004/VC-Industry-Predictive-analysis/blob/main/VC_Target_KMEANS_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing the libraries

In [25]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sn


## Importing the dataset







In [26]:
url1 = 'https://raw.githubusercontent.com/Mayank-004/Project-in-python/main/VC_Exits.csv'
dataset1 = pd.read_csv(url1, error_bad_lines=False)

#url2 = 'https://raw.githubusercontent.com/Mayank-004/Project-in-python/main/VCExit2.csv'
#dataset2 = pd.read_csv(url2, error_bad_lines=False)

url3 = 'https://raw.githubusercontent.com/Mayank-004/Project-in-python/main/VCData2.csv'
dataset3 = pd.read_csv(url3, error_bad_lines=False)

dataset = pd.merge(dataset1,dataset3,left_on=["Company PBID"],right_on=["Company ID"])



dataset


,Company PBID,Exit Date,Exit Size (millions),Pre Value (millions),Post Value (millions),Exit Type,Last VC Deal Type,Total VC Capital Raised ($M)***,Total Preferred Capital Raised ($M)*,% Preferred Capital Raised*,Total Preferred Capital Raised ($M) - Combination**,% Preferred Capital Raised - Combination**,MOIC,Industry Sector,Industry Group,Industry Code,Verticals_x,City,State,Country,Deal ID,Company ID,Primary Industry Sector,Primary Industry Group,Primary Industry Code,All Industries,Verticals_y,Keywords,Current Financing Status,Current Business Status,Universe,CEO PBId,CEO Education,Deal No.,Deal ID.1,Announced Date,Deal Date,Deal Size,Deal Size Status,Pre-money Valuation,...,# Investors,# New Investors,# Follow-on Investors,Implied EV,Revenue,Revenue Growth since last debt deal,Gross Profit,Net Income,EBITDA,EBIT,Total Debt (from financials),Fiscal Year,Valuation/EBITDA,Valuation/EBIT,Valuation/Net Income,Valuation/Revenue,Valuation/Cash Flow,Deal Size/EBITDA,Deal Size/EBIT,Deal Size/Net Income,Deal Size/Revenue,Deal Size/Cash Flow,Debt/EBITDA,Debt/Equity,Implied EV/EBITDA,Implied EV/EBIT,Implied EV/Net Income,Implied EV/Revenue,Implied EV/Cash Flow,EBITDA Margin %,Current Employees,Native Currency of Deal,HQ Location,HQ Global Region,HQ Global Sub Region,Company City,Company State/Province,Company Post Code,Company Country,Year Founded
0,155748-70,11/15/17,NaN,NaN,NaN,Merger/Acquisition,Early Stage VC,NaN,NaN,NaN,NaN,NaN,NaN,Information Technology,Software,Other Software,"3D Printing, TMT, CloudTech & DevOps",Park City,Utah,United States,67911-49T,155748-70,Information Technology,Software,Other Software,"Business/Productivity Software, Other Software*","3D Printing, CloudTech & DevOps, TMT","additives manufacturing software, additives pr...",Formerly VC-backed,Generating Revenue,Venture Capital,131029-48P,"Texas A&M University, Ph.D. (Doctor of Philoso...",1,67911-49T,NaN,01-Jan-2015,0.33,Actual,NaN,...,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,US Dollars (USD),"Park City, UT",Americas,North America,Park City,Utah,84098,United States,2014.0
1,155748-70,11/15/17,NaN,NaN,NaN,Merger/Acquisition,Early Stage VC,NaN,NaN,NaN,NaN,NaN,NaN,Information Technology,Software,Other Software,"3D Printing, TMT, CloudTech & DevOps",Park City,Utah,United States,122433-04T,155748-70,Information Technology,Software,Other Software,"Business/Productivity Software, Other Software*","3D Printing, CloudTech & DevOps, TMT","additives manufacturing software, additives pr...",Formerly VC-backed,Generating Revenue,Venture Capital,131029-48P,"Texas A&M University, Ph.D. (Doctor of Philoso...",2,122433-04T,NaN,26-May-2015,NaN,NaN,NaN,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,US Dollars (USD),"Park City, UT",Americas,North America,Park City,Utah,84098,United States,2014.0
2,160695-55,1/13/17,NaN,NaN,NaN,Merger/Acquisition,Early Stage VC,$0.70,NaN,NaN,NaN,NaN,NaN,Information Technology,Software,Software Development Applications,"SaaS, TMT, CloudTech & DevOps",Tel Aviv,NaN,Israel,71323-57T,160695-55,Information Technology,Software,Software Development Applications,"Application Software, Software Development App...","CloudTech & DevOps, SaaS, TMT","a/b testing, application development, applicat...",Formerly VC-backed,Generating Revenue,Venture Capital,56786-14P,"BA (Bachelor of Arts), 2005, Business & Marketing",1,71323-57T,NaN,18-Mar-2016,0.70,Actual,NaN,...,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,US Dollars (USD),"Tel Aviv, Israel",Middle East,Middle East,Tel Aviv,NaN,NaN,Israel,2015.0
3,81857-08,3/18/19,$166.00,NaN,$166.00,Merger/Acquisition,Early Stage VC,$10.10,NaN,NaN,NaN,NaN,16.44x,Information Technology,Software,Business/Productivity Software,"SaaS, CloudTech & DevOps",Georgetown,Texas,United States,46035

In [27]:
dataset['MOIC']

0         NaN
1         NaN
2         NaN
3      16.44x
4         NaN
        ...  
300       NaN
301       NaN
302       NaN
303    55.20x
304    55.20x
Name: MOIC, Length: 305, dtype: object

Filling missing values of Target column MOIC

In [28]:
# target column MOIC is derived by Exit Size divided by Total VC capital raised
dataset['Total VC Capital Raised ($M)***']= dataset['Total VC Capital Raised ($M)***'].str.replace("$","") # removing $ from Total VC capital
dataset['Total VC Capital Raised ($M)***']= dataset['Total VC Capital Raised ($M)***'].str.replace(",","") # removing "," from Total VC capital
dataset.loc[:,'Total VC Capital Raised ($M)***'] = dataset.loc[:,'Total VC Capital Raised ($M)***'].astype(float) # converting Total VC capital to float from string
dataset['Total VC Capital Raised ($M)***']
dataset['Exit Size (millions)']= dataset['Exit Size (millions)'].str.replace("$","") # removing $ from Exit Size
dataset['Exit Size (millions)']= dataset['Exit Size (millions)'].str.replace(",","") # removing "," from Exit Size
dataset.loc[:,'Exit Size (millions)'] = dataset.loc[:,'Exit Size (millions)'].astype(float) # converting Total VC capital to float from string

dataset['MOIC']= dataset['MOIC'].str.replace("x","") # removing $ from MOIC
dataset['MOIC']= dataset['MOIC'].str.replace(",","") # removing ',' from MOIC
dataset.loc[:,'MOIC'] = dataset.loc[:,'MOIC'].astype(float) # converting MOIC to float from string

#dataset=dataset.drop(columns=['Exit Size (millions)','Total VC Capital Raised ($M)***'])


In [5]:
#dataset.dropna(subset = ["MOIC"], inplace=True) # Drop rows with no values from Target column

In [29]:
# Drop duplicate columns  

dataset=dataset.drop(columns=['Deal ID','Company ID','Primary Industry Sector','Primary Industry Group','Primary Industry Code','All Industries','Verticals_y','HQ Location','HQ Global Region','HQ Global Sub Region','Company City',		'Company Country'])



In [30]:
# Drop Column intuitively which may not have significant impact
dataset=dataset.drop(columns=['Company PBID','Exit Date','Post Value (millions)','Native Currency of Deal','Keywords','Deal No.','Deal ID.1','Deal Date','Deal Class','Deal Status','CEO PBId'])





In [31]:
# check missing value in dataset
missing = dataset.isna().sum().sort_values(ascending = False)
percent_missing = ((missing / dataset.isnull().count()) * 100).sort_values(ascending = False)
missing_df = pd.concat([missing,percent_missing], axis = 1, keys = ['Total', 'Percent'],sort = False)
missing_df[missing_df['Total'] >= 1]

,Total,Percent
Deal Type 3,305,100.000000
Implied EV,305,100.000000
Debt Type 3,305,100.000000
Implied EV/Cash Flow,305,100.000000
Implied EV/Revenue,305,100.000000
...,...,...
Company Post Code,28,9.180328
Raised to Date,24,7.868852
Total VC Capital Raised ($M)***,23,7.540984
Year Founded,5,1.639344


Drop Column which has missing values more than 5%

---



In [32]:
missing_df = missing_df.reset_index()
over30 = list(missing_df.loc[missing_df['Percent'] >= 5,'index'])
dataset = dataset.drop(over30, axis = 1)

In [33]:
dataset

,Exit Type,Last VC Deal Type,Industry Sector,Industry Group,Industry Code,Verticals_x,City,Country,Current Financing Status,Current Business Status,Universe,Deal Type,Business Status,Financing Status,Year Founded
0,Merger/Acquisition,Early Stage VC,Information Technology,Software,Other Software,"3D Printing, TMT, CloudTech & DevOps",Park City,United States,Formerly VC-backed,Generating Revenue,Venture Capital,Grant,Generating Revenue,Corporation,2014.0
1,Merger/Acquisition,Early Stage VC,Information Technology,Software,Other Software,"3D Printing, TMT, CloudTech & DevOps",Park City,United States,Formerly VC-backed,Generating Revenue,Venture Capital,Early Stage VC,Generating Revenue,Venture Capital-Backed,2014.0
2,Merger/Acquisition,Early Stage VC,Information Technology,Software,Software Development Applications,"SaaS, TMT, CloudTech & DevOps",Tel Aviv,Israel,Formerly VC-backed,Generating Revenue,Venture Capital,Early Stage VC,Product In Beta Test,Venture Capital-Backed,2015.0
3,Merger/Acquisition,Early Stage VC,Information Technology,Software,Business/Productivity Software,"SaaS, CloudTech & DevOps",Georgetown,United States,Formerly VC-backed,Generating Revenue,Venture Capital,Early Stage VC,Generating Revenue,Venture Capital-Backed,2013.0
4,Merger/Acquisition,Seed Round,Information Technology,Software,Application Software,"Mobile, Artificial Intelligence & Machine Lear...",Minsk,Belarus,Formerly VC-backed,Startup,Venture Capital,Seed Round,Startup,Venture Capital-Backed,2016.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,IPO,Later Stage VC,Information Technology,Software,Business/Productivity Software,"CloudTech & DevOps, TMT, Supply Chain Tech",Shanghai,China,Venture Capital-Backed,Generating Revenue,"Publicly Listed, Venture Capital",Later Stage VC,Generating Revenue,Venture Capital-Backed,2001.0
301,IPO,Later Stage VC,Information Technology,Software,Business/Productivity Software,"CloudTech & DevOps, TMT, Supply Chain Tech",Shanghai,China,Venture Capital-Backed,Generating Revenue,"Publicly Listed, Venture Capital",Later Stage VC,Generating Revenue,Venture Capital-Backed,2001.0
302,IPO,Corporate,Information Technology,Computer Hardware,"Computers, Parts and Peripherals","Manufacturing, TMT, CloudTech & DevOps",Beijing,China,Formerly VC-backed,Profitable,"Publicly Listed, Venture Capital",Later Stage VC,Profitable,Venture Capital-Backed,2008.0
303,IPO,Series D,Information Technology,Software,Communication Software,"SaaS, Mobile, TMT, CloudTech & DevOps",San Jose,United States,Formerly VC-backed,Profitable,"Publicly Listed, Venture Capital",Later Stage VC,Generating Revenue,Venture Capital-Backed,2011.0


KMeans(algorithm='auto', copy_x=True, init='random', max_iter=300, n_clusters=3,
       n_init=10, n_jobs=None, precompute_distances='auto', random_state=42,
       tol=0.0001, verbose=0)

In [37]:
dataset

,Exit Type,Last VC Deal Type,Industry Sector,Industry Group,Industry Code,Verticals_x,City,Country,Current Financing Status,Current Business Status,Universe,Deal Type,Business Status,Financing Status,Year Founded
0,Merger/Acquisition,Early Stage VC,Information Technology,Software,Other Software,"3D Printing, TMT, CloudTech & DevOps",Park City,United States,Formerly VC-backed,Generating Revenue,Venture Capital,Grant,Generating Revenue,Corporation,2014.0
1,Merger/Acquisition,Early Stage VC,Information Technology,Software,Other Software,"3D Printing, TMT, CloudTech & DevOps",Park City,United States,Formerly VC-backed,Generating Revenue,Venture Capital,Early Stage VC,Generating Revenue,Venture Capital-Backed,2014.0
2,Merger/Acquisition,Early Stage VC,Information Technology,Software,Software Development Applications,"SaaS, TMT, CloudTech & DevOps",Tel Aviv,Israel,Formerly VC-backed,Generating Revenue,Venture Capital,Early Stage VC,Product In Beta Test,Venture Capital-Backed,2015.0
3,Merger/Acquisition,Early Stage VC,Information Technology,Software,Business/Productivity Software,"SaaS, CloudTech & DevOps",Georgetown,United States,Formerly VC-backed,Generating Revenue,Venture Capital,Early Stage VC,Generating Revenue,Venture Capital-Backed,2013.0
4,Merger/Acquisition,Seed Round,Information Technology,Software,Application Software,"Mobile, Artificial Intelligence & Machine Lear...",Minsk,Belarus,Formerly VC-backed,Startup,Venture Capital,Seed Round,Startup,Venture Capital-Backed,2016.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,IPO,Later Stage VC,Information Technology,Software,Business/Productivity Software,"CloudTech & DevOps, TMT, Supply Chain Tech",Shanghai,China,Venture Capital-Backed,Generating Revenue,"Publicly Listed, Venture Capital",Later Stage VC,Generating Revenue,Venture Capital-Backed,2001.0
301,IPO,Later Stage VC,Information Technology,Software,Business/Productivity Software,"CloudTech & DevOps, TMT, Supply Chain Tech",Shanghai,China,Venture Capital-Backed,Generating Revenue,"Publicly Listed, Venture Capital",Later Stage VC,Generating Revenue,Venture Capital-Backed,2001.0
302,IPO,Corporate,Information Technology,Computer Hardware,"Computers, Parts and Peripherals","Manufacturing, TMT, CloudTech & DevOps",Beijing,China,Formerly VC-backed,Profitable,"Publicly Listed, Venture Capital",Later Stage VC,Profitable,Venture Capital-Backed,2008.0
303,IPO,Series D,Information Technology,Software,Communication Software,"SaaS, Mobile, TMT, CloudTech & DevOps",San Jose,United States,Formerly VC-backed,Profitable,"Publicly Listed, Venture Capital",Later Stage VC,Generating Revenue,Venture Capital-Backed,2011.0


In [38]:
# calculating total working duration of company


from datetime import date 
todays_date = date.today() 

ts =todays_date.year

dataset['Total duration for Company'] = ts - dataset['Year Founded']

dataset=dataset.drop(columns=['Year Founded'])

dataset['Total duration for Company'] 

0       7.0
1       7.0
2       6.0
3       8.0
4       5.0
       ... 
300    20.0
301    20.0
302    13.0
303    10.0
304    10.0
Name: Total duration for Company, Length: 305, dtype: float64

In [39]:
dataset

,Exit Type,Last VC Deal Type,Industry Sector,Industry Group,Industry Code,Verticals_x,City,Country,Current Financing Status,Current Business Status,Universe,Deal Type,Business Status,Financing Status,Total duration for Company
0,Merger/Acquisition,Early Stage VC,Information Technology,Software,Other Software,"3D Printing, TMT, CloudTech & DevOps",Park City,United States,Formerly VC-backed,Generating Revenue,Venture Capital,Grant,Generating Revenue,Corporation,7.0
1,Merger/Acquisition,Early Stage VC,Information Technology,Software,Other Software,"3D Printing, TMT, CloudTech & DevOps",Park City,United States,Formerly VC-backed,Generating Revenue,Venture Capital,Early Stage VC,Generating Revenue,Venture Capital-Backed,7.0
2,Merger/Acquisition,Early Stage VC,Information Technology,Software,Software Development Applications,"SaaS, TMT, CloudTech & DevOps",Tel Aviv,Israel,Formerly VC-backed,Generating Revenue,Venture Capital,Early Stage VC,Product In Beta Test,Venture Capital-Backed,6.0
3,Merger/Acquisition,Early Stage VC,Information Technology,Software,Business/Productivity Software,"SaaS, CloudTech & DevOps",Georgetown,United States,Formerly VC-backed,Generating Revenue,Venture Capital,Early Stage VC,Generating Revenue,Venture Capital-Backed,8.0
4,Merger/Acquisition,Seed Round,Information Technology,Software,Application Software,"Mobile, Artificial Intelligence & Machine Lear...",Minsk,Belarus,Formerly VC-backed,Startup,Venture Capital,Seed Round,Startup,Venture Capital-Backed,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,IPO,Later Stage VC,Information Technology,Software,Business/Productivity Software,"CloudTech & DevOps, TMT, Supply Chain Tech",Shanghai,China,Venture Capital-Backed,Generating Revenue,"Publicly Listed, Venture Capital",Later Stage VC,Generating Revenue,Venture Capital-Backed,20.0
301,IPO,Later Stage VC,Information Technology,Software,Business/Productivity Software,"CloudTech & DevOps, TMT, Supply Chain Tech",Shanghai,China,Venture Capital-Backed,Generating Revenue,"Publicly Listed, Venture Capital",Later Stage VC,Generating Revenue,Venture Capital-Backed,20.0
302,IPO,Corporate,Information Technology,Computer Hardware,"Computers, Parts and Peripherals","Manufacturing, TMT, CloudTech & DevOps",Beijing,China,Formerly VC-backed,Profitable,"Publicly Listed, Venture Capital",Later Stage VC,Profitable,Venture Capital-Backed,13.0
303,IPO,Series D,Information Technology,Software,Communication Software,"SaaS, Mobile, TMT, CloudTech & DevOps",San Jose,United States,Formerly VC-backed,Profitable,"Publicly Listed, Venture Capital",Later Stage VC,Generating Revenue,Venture Capital-Backed,10.0


## Filling Missing Values

In [44]:
# check missing value in df
missing = dataset.isna().sum().sort_values(ascending = False)
percent_missing = ((missing / dataset.isnull().count()) * 100).sort_values(ascending = False)
missing_df = pd.concat([missing,percent_missing], axis = 1, keys = ['Total', 'Percent'],sort = False)
missing_df[missing_df['Total'] >= 1]

,Total,Percent
Financing Status,1,0.327869


In [ ]:

# filling missing values in Raised to date column with mean 

dataset['Raised to Date']= dataset['Raised to Date'].str.replace(",","")

dataset.loc[:,'Raised to Date'] = dataset.loc[:,'Raised to Date'].astype(float)

dataset['Raised to Date'] = dataset['Raised to Date'].fillna((dataset['Raised to Date'].mean()))

In [47]:
# filling missing values in all other columns with mean 

dataset = dataset.fillna((dataset.mean()))

dataset.dropna(subset = ["Financing Status"], inplace=True)
#dataset.apply(lambda x: x.fillna(x.mean()),axis=0)

In [ ]:
# check MOIC's distribution
import seaborn as sn

sn.boxplot(y=dataset["MOIC"])
plt.show()
sn.histplot(dataset, x ='MOIC')
plt.show()


# check the relationship between MOIC and Primary Industry Sector
v1 = sn.violinplot(x="Industry Sector", y="MOIC", data=dataset, size=6)
v1.set_xticklabels(v1.get_xticklabels(),rotation=90)

#Primary Industry Sector can be encoded into 2 types

# check the relationship between MOIC and Current Financing Status
v2 = sn.violinplot(x="Current Financing Status", y="MOIC", data=dataset, size=6)
v2.set_xticklabels(v2.get_xticklabels(),rotation=90)

#Current Financing Status can be encoded into 4 types

# check the relationship between MOIC and Current Business Status
v3 = sn.violinplot(x="Current Business Status", y="MOIC", data=dataset, size=6)
v3.set_xticklabels(v3.get_xticklabels(),rotation=90)

# Current Business Status can be encoded into 2 types

# check the relationship between MOIC and Universe
v4 = sn.violinplot(x="Universe", y="MOIC", data=dataset, size=6)
v4.set_xticklabels(v4.get_xticklabels(),rotation=90)

#Universe can be encoded into 4 types

# check the relationship between MOIC and VC Round
#v5 = sn.violinplot(x="VC Round", y="MOIC", data=dataset, size=6)
#v5.set_xticklabels(v5.get_xticklabels(),rotation=90)

# VC Round Status can be encoded into 2 types

# check the relationship between MOIC and Business Status
v7 = sn.violinplot(x="Business Status", y="MOIC", data=dataset, size=6)
v7.set_xticklabels(v7.get_xticklabels(),rotation=90)

#Business Status can be encoded into 3 types



In [ ]:
### Assigning 0 for MOIC less than 2, 1 for 2 to 3, 3 for 3 and above

dataset['MOIC']=pd.cut(dataset.MOIC,bins=[0,2,3,1000],labels=['0','1','2'])

## Encoding categorical data

Label Encoding of Categorical Variable

In [48]:
from sklearn.preprocessing import LabelEncoder
le= LabelEncoder()

dataset["Exit Type"]=le.fit_transform(dataset["Exit Type"])


#dataset[['Exit Type_LE']] = le.fit_transform(dataset[['Exit Type']])

dataset['Last VC Deal Type'] = le.fit_transform(dataset['Last VC Deal Type'])

dataset['Industry Sector'] = le.fit_transform(dataset['Industry Sector'])

dataset['Industry Group'] = le.fit_transform(dataset['Industry Group'])

dataset['Industry Code'] = le.fit_transform(dataset['Industry Code'])

dataset['Verticals_x'] = le.fit_transform(dataset['Verticals_x'])

dataset['City'] = le.fit_transform(dataset['City'])


dataset['Country'] = le.fit_transform(dataset['Country'])

dataset['Current Financing Status'] = le.fit_transform(dataset['Current Financing Status'])
dataset['Current Business Status'] = le.fit_transform(dataset['Current Business Status'])
dataset['Universe'] = le.fit_transform(dataset['Universe'])
dataset['Business Status'] = le.fit_transform(dataset['Business Status'])
dataset['Financing Status'] = le.fit_transform(dataset['Financing Status'])
dataset['Deal Type'] = le.fit_transform(dataset['Deal Type'])

dataset

,Exit Type,Last VC Deal Type,Industry Sector,Industry Group,Industry Code,Verticals_x,City,Country,Current Financing Status,Current Business Status,Universe,Deal Type,Business Status,Financing Status,Total duration for Company
0,2,1,4,9,21,0,47,16,2,0,10,4,0,2,7.0
1,2,1,4,9,21,0,47,16,2,0,10,2,0,4,7.0
2,2,1,4,9,23,53,66,10,2,0,10,2,3,4,6.0
3,2,1,4,9,2,39,27,16,2,0,10,2,0,4,8.0
4,2,3,4,9,0,24,41,1,2,4,10,6,5,4,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,1,2,4,9,2,12,62,4,5,0,9,5,0,4,20.0
301,1,2,4,9,2,12,62,4,5,0,9,5,0,4,20.0
302,1,0,4,2,4,22,5,4,2,3,9,5,4,4,13.0
303,1,12,4,9,3,51,57,16,2,3,9,5,0,4,10.0


Changing Data type of Column to perform correlation matrix

Removal of The columns which are highly related to other columns

### Building Models

In [ ]:
 import matplotlib.pyplot as plt
 #from kneed import kneeLocator
 from sklearn.cluster import KMeans
 from sklearn.metrics import silhouette_score
 from sklearn.preprocessing import StandardScaler

In [ ]:
kmeans = KMeans(
            init="random",
            n_clusters=3,
            n_init=10,
            max_iter=300,
            random_state=42
  )

In [52]:
kmeans.fit(dataset)

KMeans(algorithm='auto', copy_x=True, init='random', max_iter=300, n_clusters=3,
       n_init=10, n_jobs=None, precompute_distances='auto', random_state=42,
       tol=0.0001, verbose=0)

In [55]:
kmeans.cluster_centers_


array([[ 1.60759494,  5.57594937,  3.84810127,  8.12025316, 10.74683544,
        49.53164557, 54.4556962 , 15.25316456,  2.31012658,  0.49367089,
         8.70886076,  3.62658228,  0.98101266,  3.7278481 ,  9.96820675],
       [ 1.50561798,  6.28089888,  3.97752809,  8.57303371,  8.5505618 ,
        35.35955056, 12.73033708, 11.24719101,  2.47191011,  0.4494382 ,
         7.93258427,  3.14606742,  1.03370787,  3.53932584,  9.47138577],
       [ 1.71929825,  5.21052632,  3.75438596,  8.01754386,  8.77192982,
        17.42105263, 54.52631579, 12.84210526,  2.42105263,  0.50877193,
         8.28070175,  3.77192982,  1.29824561,  3.9122807 , 10.64830409]])

In [54]:
from kmodes.kmodes import KModes
KModes(n_clusters=2, init = "Cao", n_init = 1, verbose=1)

ModuleNotFoundError: ignored

In [ ]:
km_huang = KModes(n_clusters=2, init = "Huang", n_init = 1, verbose=1)
fitClusters_huang = km_huang.fit_predict(dataset)
# Predicted clusters
fitClusters_huang